In [ ]:
'''
Created By Josh Stokley to create graph of cracking angle in concrete of each integration point
The script provides-
variables:
 refFile ........................... Defined txt file with variables related to RC wall
 ip1sig ............................ stress integration point 1 data
 ip2sig ............................ stress integration point 2 data 
 levels ............................ total levels of elements up the wall
 ft ................................ tensile strength of concrete
 tclFileName ....................... file path of tcl file
 x_nodes ........................... location of nodes in x direction
 minum ............................. minimum length of simulated files
 sig ............................... array of stress values for each integration point (holds stress in x, y and tau)
 orient ............................ orientation of crack for each integration point
 tot_x ............................. location of nodes in x direction
 tot_y ............................. location of nodes in y direction
 length ............................ length of elements
 x_loc ............................. location of integration points in x direction
 
 
methods:
 def getCrackedModel(ip1sig, ip2sig,refFile) ....... imports reference file and stress/strain data
 def findOrien(ip1sig,ip2sig) ...................... returns values of stress in x, y, and tau at concrete cracking point
 def ellipse ....................................... creates thin ellipses at integration points to demonstrate cracking angle
 location script section ........................... finds locations of integration points in x direction
 plotting script section ........................... plots ellipses at integration points where the angle of each ellipse is angle of the crack
'''


import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import io
import sys
import math

def getCrackedModel(ip1sig, ip2sig,refFile):
    
    with open(refFile, 'rt') as h:
        data = [line.split() for line in h]
        tot_eles = int(data[3][0])
        ft = float(data[5][0])
        levels = int(data[6][0])
        filepath = data[8][0]
        tclFileName = data[9][0]

    with open(tclFileName, 'rt') as h:
        data = [line.split() for line in h]
        found = False
        x_nodes = []
        for ii in range(len(data)):
            if len(data[ii]) > 1  and data[ii][0] == 'node' and found == False:
                if len(x_nodes)> 1 and (float(data[ii][2])) == x_nodes[0] and found == False:
                    y_inc = float(data[ii][3])
                    nodes = int(data[ii-2][1])
                    found = True
                if found == False:
                    x_nodes.append(float(data[ii][2]))
    
    def angle(filename,maxEle):
        with open(filename, 'r') as h:
            data = [line.split() for line in h]
            for ii in range(len(data)):
                if data[ii] == ['<Data>']:
                    start = ii + 1

            string = True 
            k = -1
            back = 0
            while string == True:
                if len(data[k])< maxEle*2:
                    back +=1
                    k -=1
                else:
                    string = False

            crackAngle = data[k]
            for ii in range(len(crackAngle)):
                crackAngle[ii] = float(crackAngle[ii])

            crackAngle = crackAngle[2::3] 
        return crackAngle
                
    angle1 = angle(ip1sig,tot_eles)
    angle2 = angle(ip2sig,tot_eles)
    


    def ellipse(x,y,orient):
        orient = float(orient)
        import numpy as np
        from matplotlib import pyplot as plt
        from math import pi, cos, sin

        u=x          #x-position of the center
        v=y          #y-position of the center
        a=1          #radius on the x-axis
        b=.025       #radius on the y-axis
        t_rot=orient*0.0174533 #rotation angle

        t = np.linspace(0, 2*pi, 100)
        Ell = np.array([a*np.cos(t) , b*np.sin(t)])  
             #u,v removed to keep the same center location
        R_rot = np.array([[cos(t_rot) , -sin(t_rot)],[sin(t_rot) , cos(t_rot)]])  
             #2-D rotation matrix

        Ell_rot = np.zeros((2,Ell.shape[1]))
        for i in range(Ell.shape[1]):
            Ell_rot[:,i] = np.dot(R_rot,Ell[:,i])

        x_fin = u+Ell_rot[0,:]
        y_fin = v+Ell_rot[1,:]

        return x_fin, y_fin


    y_list =  []

    for ii in range(levels+1):
        y_list.append(ii*y_inc)

    tot_x = []
    tot_y = []
    for ii in range(len(y_list)):
        for jj in range(len(x_nodes)):
            tot_x.append(x_nodes[jj])
            tot_y.append(y_list[ii])
    
    x = x_nodes
    length = (x[1] - x[0])/2
    x_loc = []
    for ii in range(nodes):
        if x[ii] == 0:
            x_loc.append(length*1/3)
        elif x[ii] == x[-1]:
            x_loc.append(x[-1] - (length*2/3))
        else:
            x_loc.append(x[ii] - length*1/3)
            x_loc.append(x[ii] + length*1/3)

    orient = []
    for ii in range(len(angle1)):
        orient.append(angle1[ii])
        orient.append(angle2[ii])
        
    for ii in range(len(orient),len(x_loc)):
        orient.append(0)
    
    print(orient[:30])
    fig, ax = plt.subplots(figsize=(15, 30),linewidth=4, edgecolor="#04253a")

    for ii in range(len(y_list)):
        y_loc = []
        for jj in range(len(x_nodes)):
            y_loc.append(y_list[ii])

        ax.plot(x_nodes,y_loc, color = 'gray')

    for ii in range(len(x_nodes)):
        x = [x_nodes[ii],x_nodes[ii]]
        y = [y_list[0],y_list[-1]]

        ax.plot(x,y, color = 'gray')

    level = 0
    location = 0
    for ii in range(len(orient)):
        x,y = ellipse(x_loc[location],y_list[level],orient[ii])
        location +=1
        if location == len(x_loc):
            level +=1
            location = 0
        if float(orient[ii] ) < 90:
            col = 'blue'
        elif float(orient[ii] ) > 90:
            col = 'red'
        elif orient[ii] == 0:
            col = 'black'
        ax.plot(x,y,color = 'black')

    fig.savefig(filepath+'cracked.png')
    #plt.close(fig)

In [ ]:
files = [
'tcl_files/tcl_files_23022600/DazioWSH3_230226004710/230226004710referenceFile.txt'
]


In [ ]:
for ii in files:
    loc = ii[:-29]
    filename = loc + 'Crack_elementsmat1fib3crack.xml'
    filename1 = loc + 'Crack_elementsmat2fib3crack.xml'
    getCrackedModel(filename, filename1,ii)